In [116]:
import mpmath as mp
mp.dps = 30; mp.pretty = True
from mpmath import findroot as findroot_mp

In [9]:
# defining instance
import numpy as np
from matplotlib import lines
import matplotlib.pyplot as plt
import copy
import time
from scipy.optimize import fsolve, fmin
import math
from itertools import combinations


from KP import KnapsackProblem

### Root Finding Algorithms and Computing $\gamma$

July 20, 2022

Review some root finding algorithms for the purpose of computing $z_0$ and $\lim_{\beta \to \infty} \beta^{-1} \ln z_0$ for the Knapsack problem.

In [3]:
items = (
    ("map", 9, 150), ("compass", 13, 35), ("water", 153, 200), ("sandwich", 50, 160),
    ("glucose", 15, 60), ("tin", 68, 45), ("banana", 27, 60), ("apple", 39, 40),
    ("cheese", 23, 30), ("beer", 52, 10), ("suntan cream", 11, 70), ("camera", 32, 30),
    ("t-shirt", 24, 15), ("trousers", 48, 10), ("umbrella", 73, 40),
    ("waterproof trousers", 42, 70), ("waterproof overclothes", 43, 75),
    ("note-case", 22, 80), ("sunglasses", 7, 20), ("towel", 18, 12),
    ("socks", 4, 50), ("book", 30, 10),
    )

factor = 1.0

# defining weight and value vectors and weight limit
weight_vec = np.array([item[1]/factor for item in items])
value_vec = np.array([item[2]/factor for item in items])
Wlimit = 400/factor


In [8]:
z0, T0 = 0.5, 0.5

# mp definition of exponential
mp_exp_array = np.frompyfunc(mp.exp, 1, 1)

# constraint function
constraint = lambda z, T,  weights, values, limit: -limit+ z/(1-z) + np.sum(weights/(mp_exp_array(-values/T)*mp_exp_array(-weights*mp.log(z)) + 1))

# testing constraint function
constraint(z0, T0, weight_vec, value_vec, Wlimit)

mpf('283.33651395650634')

In [152]:
# constraint function
deriv_constraint = lambda z, T,  weights, values, limit: 1/(1-z)**2 + (1/z)*np.sum(weights**2*mp_exp_array(-values/T)*mp_exp_array(-weights*mp.log(z))/(mp_exp_array(-values/T)*mp_exp_array(-weights*mp.log(z)) + 1)**2)


In [169]:
z0 = 0.01

In [170]:
constraint(z0, T0, weight_vec, value_vec, Wlimit)

mpf('-268.64579280063077')

In [171]:
deriv_constraint(z0, T0, weight_vec, value_vec, Wlimit)

mpf('930.1977783375338')

#### Root Finding Through Newton's 

In [190]:
def find_root_newtons(func, deriv_func, verbose = False, etol = 1e-6, z=1e-9, **args):

    """
    Finds the root of a function in the domain (0, 1) by 
    the Newtons method

    Parameters
    ----------
    func : function
        Function for which we are seeking the root
        
    num_powers : int
        Number of powers of 10 to include in final answer

    Returns
    ----------
    z : float
        Value of critical point

    error : float 
        How far function is from zero at this critical point

    """        
    
    error = np.abs(func(z, **args).real)
    print(z)
    
    iter_ = 0
    while error > etol:
        h = func(z, **args).real/(deriv_func(z, **args).real)        
        z = z - h
        if verbose:
            print(z)
        error = np.abs(func(z, **args).real)
        iter_+=1
        if iter_ == 100:
            print('Method is failing converge; Breaking')
            break

    return z, error  

#### Root Finding Through Bijection

In [55]:
def find_root_bijection(func, num_bijections = 50, **args):

    """
    Finds the root of a function in the domain (0, 1) by 
    the bijection method

    Parameters
    ----------
    func : function
        Function for which we are seeking the root
        
    num_powers : int
        Number of powers of 10 to include in final answer

    Returns
    ----------
    fin_z : float
        Value of critical point

    error : float 
        How far function is from zero at this critical point

    """        
    
    a = 1e-9
    b = 1-1e-9
    c = (a+b)/2

    for k in range(num_bijections):
        f_a = func(a, **args).real
        f_b = func(b, **args).real
        f_c = func(c, **args).real

        if np.sign(f_a) == -np.sign(f_c):
            b = c
            c = (a+b)/2

        elif np.sign(f_b) == -np.sign(f_c):  
            a = c
            c = (a+b)/2

    fin_z = c
    error = func(c, **args).real

    return fin_z, error  

#### Root Finding through False Position

In [78]:
def find_root_falseposition(func, num_bijections = 50, **args):

    """
    Finds the root of a function in the domain (0, 1) by 
    the false position method

    Parameters
    ----------
    func : function
        Function for which we are seeking the root
        
    num_powers : int
        Number of powers of 10 to include in final answer

    Returns
    ----------
    fin_z : float
        Value of critical point

    error : float 
        How far function is from zero at this critical point

    """        
    
    a = 1e-9
    b = 1-1e-9
    f_a = func(a, **args).real
    f_b = func(b, **args).real
    
     
    c = (a*f_b-b*f_a)/(f_b-f_a)
    f_c = func(c, **args).real   

    for k in range(num_bijections):
        f_a = func(a, **args).real
        f_b = func(b, **args).real
        f_c = func(c, **args).real

        if np.sign(f_a) == -np.sign(f_c):
            b = c
            f_b = func(b, **args).real
            
            c = (a*f_b-b*f_a)/(f_b-f_a)

        elif np.sign(f_b) == -np.sign(f_c):  
            a = c
            f_a = func(a, **args).real
            
            c = (a*f_b-b*f_a)/(f_b-f_a)

    fin_z = c
    error = func(c, **args).real

    return fin_z, error  

#### Root Finding Through Interpolation

In [56]:
def find_root(func, num_powers = 20, **args):
    
    """
    Finds the root of a function in the domain (0, 1) exclusive

    Parameters
    ----------
    func : function
        Function for which we are seeking the root
        
    num_powers : int
        Number of powers of 10 to include in final answer

    Returns
    ----------
    fin_z : float
        Value of critical point

    error : float 
        How far function is from zero at this critical point

    """    
    
    powers_10 = np.array([10**(-k) for k in range(num_powers+1)])
    coeffs = np.zeros_like(powers_10)
    coeffs[0] = 0

    # defining $l_start
    l_start = 1
    
    # determining initial order of current z
    for i in range(1, num_powers+1):
        coeffs[i] = 1
        current_z =  np.dot(coeffs, powers_10)
        current_sign = np.sign(func(current_z, **args).real)
        # reset coeffs if sign doesn't change
        if current_sign > 0:
            coeffs[i] = 0
        else:
            l_start = i
            break
            
    # determine subsequent coefficients
    for k in range(l_start,num_powers+1):

        # marching up
        for j in range(10):

            # try a value of the coefficient
            coeffs[k] = j
            new_z = np.dot(coeffs, powers_10)
            new_sign = np.sign(func(new_z,**args).real)  

            # if the sign has changed from the current sign, go to previous value
            if new_sign > 0:
                coeffs[k] = j-1 # set to the j value before the change
                current_z = np.dot(coeffs, powers_10)
                break

    # final z value
    fin_z = np.dot(coeffs, powers_10)    
    
    # error 
    error = func(fin_z, **args).real
    
    return fin_z, float(error)

### Example: $f(x) = - \cos(\pi x)$

In [97]:
cosine_func = lambda x: -np.cos(np.pi*(1.02345678910)*x)

deriv_cosine_func = lambda x: np.pi*(1.02345678910)*np.sin(np.pi*(1.02345678910)*x)

In [181]:
start_time = time.time()

root, error = find_root(cosine_func)
print('Root and error', root, error)
print('Elapsed Time: {:2f} secs'.format( time.time()-start_time))

Root and error 0.48854041062123044 -6.123233995736766e-17
Elapsed Time: 0.003138 secs


In [182]:
start_time = time.time()

root, error = find_root_bijection(cosine_func)
print('Root and error', root, error)
print('Elapsed Time: {:2f} secs'.format( time.time()-start_time))

Root and error 0.4885404106212309 1.4930798945178516e-15
Elapsed Time: 0.002098 secs


In [183]:
start_time = time.time()

root, error = find_root_falseposition(cosine_func)
print('Root and error', root, error)
print('Elapsed Time: {:2f} secs'.format( time.time()-start_time))

Root and error 0.4885404106212305 1.6081226496766366e-16
Elapsed Time: 0.001972 secs


In [184]:
start_time = time.time()

root, error = find_root_newtons(cosine_func, deriv_cosine_func,  z=.2)
print('Root and error', root, error)
print('Elapsed Time: {:2f} secs'.format( time.time()-start_time))

0.2
Root and error 0.4885404106212304 6.123233995736766e-17
Elapsed Time: 0.000954 secs


### Desired Case: Time Comparison

In [83]:
start_time = time.time()

root, error = find_root(constraint, T=T0, weights=weight_vec, values=value_vec, limit=Wlimit)
print('Root and error', root, error)
print('Elapsed Time: {:2f} secs'.format( time.time()-start_time))

Root and error 0.07454095467262639 -5.115907697472721e-13
Elapsed Time: 0.080367 secs


In [82]:
start_time = time.time()

root, error = find_root_bijection(constraint, T=T0, weights=weight_vec, values=value_vec, limit=Wlimit)
print('Root and error', root, error)
print('Elapsed Time: {:2f} secs'.format( time.time()-start_time))

Root and error 0.07454095467262623 -3.80850906367414e-12
Elapsed Time: 0.084775 secs


In [81]:
start_time = time.time()

root, error = find_root_falseposition(constraint, T=T0, weights=weight_vec, values=value_vec, limit=Wlimit)
print('Root and error', root, error)
print('Elapsed Time: {:2f} secs'.format( time.time()-start_time))

Root and error 1.92679751664715e-5 -375.999980744446
Elapsed Time: 0.103874 secs


In [191]:
start_time = time.time()

root, error = find_root_newtons(constraint, deriv_constraint, T=T0, weights=weight_vec, values=value_vec, limit=Wlimit)
print('Root and error', root, error)
print('Elapsed Time: {:2f} secs'.format( time.time()-start_time))

1e-09
Method is failing converge; Breaking
Root and error -1.10568132678751e+63406125634487632306126538719 402.0
Elapsed Time: 0.258661 secs
